In [1]:
import numpy as np
import pandas as pd
import copy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  cross_val_score
from sklearn.linear_model import LogisticRegression
import time

df_train=pd.read_csv('C:/Users/User/Documents/titanic_train.csv')
df_test=pd.read_csv('C:/Users/User/Documents/titanic_test.csv')

train_y=df_train['Survived']
ids=df_test['PassengerId']
df_trian=df_train.drop(['PassengerId','Survived'],axis=1)
df_test=df_test.drop(['PassengerId'],axis=1)
df=pd.concat([df_train,df_test])
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2.0,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3.0,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4.0,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5.0,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 只取出 int64, float64欄位資料，存於box_feature中
box_feature=[]
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype=='object':
        box_feature.append(feature)
print(f'{len(box_feature)} Numeric feature : {box_feature}\n')


5 Numeric feature : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



In [3]:
# 削減文字型欄位，只留數值型
df=df[box_feature]
df=df.fillna('None')
train_num=train_y.shape[0]
df.head()


,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [4]:
# 對照組: label encoding + rogistic regression
df_temp=pd.DataFrame()
for i in df.columns:
#     迭代的方式進行，並寫回原本的column中
    df_temp[i]=LabelEncoder().fit_transform(df[i])
train_x=df_temp[:train_num]
estimator=LogisticRegression()
start=time.time()
print(f'shape:{train_x.shape}')
print(f'score:{cross_val_score(estimator,train_x,train_y,cv=5).mean()}')
print(f'time:{time.time()-start} sec')

shape:(891, 5)
score:0.7800138095537004
time:0.11300039291381836 sec


In [5]:
# 實驗組 軍職編碼 + rogitic regression
data=pd.concat([df[:train_num],train_y],axis=1) #和target value 擺在一起摨能進行均值編碼
for i in df.columns:
#     迭代的方式進行，並寫回原本的column中
    mean_df=data.groupby([i])['Survived'].mean().reset_index()
    mean_df.columns=[i,f'{i}_mean']
    data=pd.merge(data,mean_df,on=i,how='left')
    data=data.drop([i],axis=1)
data=data.drop(['Survived'],axis=1)
estimator=LogisticRegression()
start=time.time()
print(f'shape:{train_x.shape}')
print(f'score:{cross_val_score(estimator,train_x,train_y,cv=5).mean()}')
print(f'time:{time.time()-start} sec') 

shape:(891, 5)
score:0.7800138095537004
time:0.08273172378540039 sec
